# Chargement des données depuis la BDD

In [5]:
import sqlite3

#Connection à la base de données
conn = sqlite3.connect('../src/videos_youtube.db')
cursor = conn.cursor()

# Charger l'url
cursor.execute(f"SELECT id, video_id, subtitle FROM video_chapters")
results = cursor.fetchall()

# Fermer la connexion à la base de données
conn.close()

# Embeddings des chapitres

In [ ]:
from mistralai import Mistral

api_key = ""

# Initialiser le client Mistral
client = Mistral(api_key=api_key)

def generate_embedding(text : str):
    """Génère un embedding"""
    # Spécifier le modèle d'embedding
    model = "mistral-embed"

    # Appeler l'API pour générer les embeddings
    response = client.embeddings.create(
        model=model,
        inputs=[text]
    )

    # Extraire les embeddings de la réponse
    prompt_embedding = response.data[0].embedding
    return prompt_embedding

In [22]:
import time

# Extraire uniquement les titres (le troisième élément de chaque tuple)
chapters = [chap for _, _, chap in results]

chap_embed = []

for chap in chapters:
    chap_embed.append(generate_embedding(chap))
    time.sleep(2)


In [25]:
#Sauvegarde des embeddings
import json
with open('chap_embeddings.json', 'w') as f:
    json.dump(chap_embed, f)

# Création de l'index Faiss pour les chapitres 

In [7]:
import faiss
import numpy as np

#Générer la liste des nouveaux IDs
new_ids = [f"{grp:02}{id_:03}" for id_, grp, _ in results]

#Définir la dimension des vecteurs 
dimension = 1024

#Crée l'index Faiss
index = faiss.IndexFlatL2(dimension)

# Ajouter les IDs des chapitres à l'index FAISS
index_with_ids = faiss.IndexIDMap(index)  # Permet d'ajouter des IDs personnalisés
index_with_ids.add_with_ids(np.array(chap_embed, dtype=np.float32), np.array(new_ids, dtype=np.int64))

faiss.write_index(index_with_ids, "faiss_index_chapters.bin")  # Sauvegarde